In [ ]:
import os
import cv2
import torch
from PIL import Image
from torchvision import transforms
from detectron2.config import get_cfg
from detectron2.engine import DefaultPredictor
from detectron2.utils.visualizer import Visualizer
from detectron2.data import MetadataCatalog

def load_model(config_file, weights_file):
    # Load the configuration
    cfg = get_cfg()
    cfg.merge_from_file(config_file)
    cfg.MODEL.WEIGHTS = weights_file
    cfg.MODEL.ROI_HEADS.SCORE_THRESH_TEST = 0.97
      # Set a custom testing threshold
    return DefaultPredictor(cfg)

def process_image(predictor, image_path):
    # Load image
    image = cv2.imread(image_path)
    outputs = predictor(image)
    
    # Visualize the results
    v = Visualizer(image[:, :, ::-1], MetadataCatalog.get(predictor.cfg.DATASETS.TRAIN[0]), scale=1.2)
    out = v.draw_instance_predictions(outputs["instances"].to("cpu"))
    
    # Display the image
    result_image = out.get_image()[:, :, ::-1]
    cv2.imshow(f'Result for {image_path}', result_image)
    cv2.waitKey(0)
    cv2.destroyAllWindows()

def process_video(predictor, video_path):
    cap = cv2.VideoCapture(video_path)
    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break
        
        outputs = predictor(frame)
        v = Visualizer(frame[:, :, ::-1], MetadataCatalog.get(predictor.cfg.DATASETS.TRAIN[0]), scale=0.4)
        out = v.draw_instance_predictions(outputs["instances"].to("cpu"))
        
        # Display the frame
        result_frame = out.get_image()[:, :, ::-1]
        cv2.imshow(f'Result for a frame in {video_path}', result_frame)
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break
    cap.release()
    cv2.destroyAllWindows()

def process_folder(predictor, folder_path):
    for file_name in os.listdir(folder_path):
        file_path = os.path.join(folder_path, file_name)
        if os.path.isfile(file_path):
            if file_path.lower().endswith(('.png', '.jpg', '.jpeg', '.bmp', '.tif', '.tiff')):
                process_image(predictor, file_path)
            elif file_path.lower().endswith(('.mp4', '.avi', '.mov', '.mkv')):
                process_video(predictor, file_path)

# Example usage
config_file = "C:/Users/Daanish Mittal/OneDrive/Desktop/Tank_align/tank_alignment/army_ramp/loader.ramp/mask_rcnn_R_101_FPN_3x/2024-07-09-01-55-42/config.yaml"
weights_file = "C:/Users/Daanish Mittal/OneDrive/Desktop/Tank_align/tank_alignment/army_ramp/loader.ramp/mask_rcnn_R_101_FPN_3x/2024-07-09-01-55-42/model_final.pth"
input_folder = "C:/Users/Daanish Mittal/OneDrive/Desktop/Tank_align/tank_alignment/test_army"


predictor = load_model(config_file, weights_file)
process_folder(predictor, input_folder)


In [22]:
import os
import cv2
import torch
from PIL import Image
from torchvision import transforms
from detectron2.config import get_cfg
from detectron2.engine import DefaultPredictor
from detectron2.utils.visualizer import Visualizer
from detectron2.data import MetadataCatalog
import numpy as np

def load_model(config_file, weights_file):
    # Load the configuration
    cfg = get_cfg()
    cfg.merge_from_file(config_file)
    cfg.MODEL.WEIGHTS = weights_file
    cfg.MODEL.ROI_HEADS.SCORE_THRESH_TEST = 0.99  # Set a custom testing threshold
    return DefaultPredictor(cfg)

def draw_quadrilaterals(image, outputs, deformation_threshold=0.05):
    masks = outputs["instances"].pred_masks.to("cpu").numpy()
    for mask in masks:
        # Find contours
        contours, _ = cv2.findContours(mask.astype(np.uint8), cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
        
        if len(contours) == 0:
            continue
        
        for contour in contours:
            # Approximate the contour to a quadrilateral
            epsilon = deformation_threshold * cv2.arcLength(contour, True)
            approx = cv2.approxPolyDP(contour, epsilon, True)
            
            # Ensure it's a quadrilateral (4 sides)
            if len(approx) == 4:
                # Draw the quadrilateral
                cv2.polylines(image, [approx], True, (0, 255, 0), 2)
                
    return image


def process_image(predictor, image_path):
    # Load image
    image = cv2.imread(image_path)
    outputs = predictor(image)
    
    # Draw quadrilaterals and masks on the image
    result_image = draw_quadrilaterals(image, outputs)
    
    # Display the image
    cv2.imshow(f'Result for {image_path}', result_image)
    cv2.waitKey(0)
    cv2.destroyAllWindows()

def process_video(predictor, video_path):
    cap = cv2.VideoCapture(video_path)
    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break
        
        outputs = predictor(frame)
        result_frame = draw_quadrilaterals(frame, outputs)
        
        # Display the frame
        cv2.imshow(f'Result for a frame in {video_path}', result_frame)
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break
    cap.release()
    cv2.destroyAllWindows()

def process_folder(predictor, folder_path):
    for file_name in os.listdir(folder_path):
        file_path = os.path.join(folder_path, file_name)
        if os.path.isfile(file_path):
            if file_path.lower().endswith(('.png', '.jpg', '.jpeg', '.bmp', '.tif', '.tiff')):
                process_image(predictor, file_path)
            elif file_path.lower().endswith(('.mp4', '.avi', '.mov', '.mkv')):
                process_video(predictor, file_path)

# Example usage
config_file = "C:/Users/Daanish Mittal/OneDrive/Desktop/Tank_align/tank_alignment/army_ramp/loader.ramp/mask_rcnn_R_101_FPN_3x/2024-07-09-01-55-42/config.yaml"
weights_file = "C:/Users/Daanish Mittal/OneDrive/Desktop/Tank_align/tank_alignment/army_ramp/loader.ramp/mask_rcnn_R_101_FPN_3x/2024-07-09-01-55-42/model_final.pth"
input_folder = "C:/Users/Daanish Mittal/OneDrive/Desktop/Tank_align/tank_alignment/test_army"

predictor = load_model(config_file, weights_file)
process_folder(predictor, input_folder)


**Calculated Code**

In [26]:
import os
import cv2
import torch
import numpy as np
from PIL import Image
from torchvision import transforms
from detectron2.config import get_cfg
from detectron2.engine import DefaultPredictor
from detectron2.utils.visualizer import Visualizer
from detectron2.data import MetadataCatalog

def load_model(config_file, weights_file):
    cfg = get_cfg()
    cfg.merge_from_file(config_file)
    cfg.MODEL.WEIGHTS = weights_file
    cfg.MODEL.ROI_HEADS.SCORE_THRESH_TEST = 0.99
    return DefaultPredictor(cfg)

def calculate_polygon_angle(contour):
    rect = cv2.minAreaRect(contour)
    angle = rect[2]
    if angle < -45:
        angle = 90 + angle
    elif angle > 45:
        angle = angle - 90
    else:
        angle = 90 - abs(angle)
    return angle

def mask_sides(image, mask_percentage=0.2):
    h, w = image.shape[:2]
    mask_width = int(w * mask_percentage)
    
    mask = np.ones(image.shape[:2], dtype=np.uint8)
    mask[:, :mask_width] = 0
    mask[:, -mask_width:] = 0
    
    masked_image = image.copy()
    masked_image[:, :mask_width] = 0
    masked_image[:, -mask_width:] = 0
    
    return masked_image, mask

def point_line_distance(point, line_start, line_end):
    line_vec = np.array(line_end) - np.array(line_start)
    point_vec = np.array(point) - np.array(line_start)
    line_len = np.linalg.norm(line_vec)
    line_unitvec = line_vec / line_len
    point_vec_scaled = point_vec / line_len
    t = np.dot(line_unitvec, point_vec_scaled)
    t = np.clip(t, 0, 1)
    nearest = line_start + t * line_vec
    dist = np.linalg.norm(point - nearest)
    return dist

def process_frame(predictor, frame, ramp_index):
    masked_frame, frame_mask = mask_sides(frame, 0.3)
    
    outputs = predictor(masked_frame)
    instances = outputs["instances"].to("cpu")
    ramp_instances = instances[instances.pred_classes == ramp_index]
    
    draw_frame = masked_frame.copy()
    
    for i in range(len(ramp_instances)):
        if ramp_instances.has("pred_masks"):
            mask = ramp_instances.pred_masks[i].numpy()
            mask = cv2.bitwise_and(mask.astype(np.uint8), frame_mask)
            
            contours, _ = cv2.findContours(mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
            if contours:
                largest_contour = max(contours, key=cv2.contourArea)
                
                epsilon = 0.05 * cv2.arcLength(largest_contour, True)
                approx_contour = cv2.approxPolyDP(largest_contour, epsilon, True)
                
                if len(approx_contour) > 4:
                    approx_contour = cv2.convexHull(approx_contour)
                    approx_contour = cv2.approxPolyDP(approx_contour, epsilon, True)
                
                if len(approx_contour) > 4:
                    approx_contour = cv2.approxPolyDP(approx_contour, epsilon * 2, True)
                
                if len(approx_contour) > 4:
                    approx_contour = cv2.convexHull(approx_contour, returnPoints=False)
                    approx_contour = cv2.approxPolyDP(approx_contour, epsilon * 3, True)
                
                approx_contour = approx_contour[:4]

                cv2.drawContours(draw_frame, [approx_contour], 0, (128, 0, 128), 2)
                
                x, y, w, h = cv2.boundingRect(largest_contour)
                cv2.rectangle(draw_frame, (x, y), (x + w, y + h), (255, 0, 0), 2)
                
                extension_length = 50
                extended_edge_top_start = (x, y)
                extended_edge_bottom_start = (x, y + h)
                extended_edge_top_end = (x, y - extension_length)
                extended_edge_bottom_end = (x, y + h + extension_length)
                
                cv2.line(draw_frame, extended_edge_top_start, extended_edge_bottom_start, (0, 0, 255), 2)
                cv2.line(draw_frame, extended_edge_top_start, extended_edge_top_end, (0, 0, 255), 2)
                cv2.line(draw_frame, extended_edge_bottom_start, extended_edge_bottom_end, (0, 0, 255), 2)
                
                ramp_edge_start = tuple(approx_contour[0][0])
                ramp_edge_end = tuple(approx_contour[1][0])
                
                vector_extended = np.array([extended_edge_bottom_end[0] - extended_edge_top_end[0], extended_edge_bottom_end[1] - extended_edge_top_end[1]])
                vector_ramp = np.array([ramp_edge_end[0] - ramp_edge_start[0], ramp_edge_end[1] - ramp_edge_start[1]])
                
                dot_product = np.dot(vector_extended, vector_ramp)
                norm_extended = np.linalg.norm(vector_extended)
                norm_ramp = np.linalg.norm(vector_ramp)
                
                cosine_angle = dot_product / (norm_extended * norm_ramp)
                angle = np.arccos(cosine_angle)
                angle_degrees = np.degrees(angle)
                
                cv2.putText(draw_frame, f"Angle: {angle_degrees:.2f}", (x, y - 10), 
                            cv2.FONT_HERSHEY_SIMPLEX, 0.9, (0, 255, 255), 2)
                
                mid_x = draw_frame.shape[1] // 2
                
                intersection_point = (mid_x, y + h)
                
                line_length = 200
                end_x = int(intersection_point[0] + line_length * np.sin(np.radians(angle_degrees)))
                end_y = int(intersection_point[1] - line_length * np.cos(np.radians(angle_degrees)))
                
                cv2.line(draw_frame, intersection_point, (end_x, end_y), (0, 255, 0), 2)

                left_edge_start = (0, 0)
                left_edge_end = (0, draw_frame.shape[0])
                
                dist = point_line_distance((0, y + h), left_edge_start, left_edge_end)
                
                if dist < 5:
                    tint_color = (0, 255, 0)  # Green
                elif dist < 20:
                    tint_color = (0, 165, 255)  # Orange
                else:
                    tint_color = (0, 0, 255)  # Red
                
                overlay = np.full(draw_frame.shape, tint_color, dtype=np.uint8)
                alpha = 0.3
                
                draw_frame = cv2.addWeighted(overlay, alpha, draw_frame, 1 - alpha, 0)

                cv2.line(draw_frame, intersection_point, (end_x, end_y), tint_color, 2)
    
    return draw_frame

def process_image(predictor, image_path, ramp_index):
    image = cv2.imread(image_path)
    result_image = process_frame(predictor, image, ramp_index)
    cv2.imshow(f'Result for {image_path}', result_image)
    cv2.waitKey(0)
    cv2.destroyAllWindows()

def process_video(predictor, video_path, ramp_index, output_path):
    cap = cv2.VideoCapture(video_path)
    fourcc = cv2.VideoWriter_fourcc(*'mp4v')
    frame_width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
    frame_height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
    out = cv2.VideoWriter(output_path, fourcc, 30.0, (frame_width, frame_height))

    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break
        
        result_frame = process_frame(predictor, frame, ramp_index)
        out.write(result_frame)
        
        cv2.imshow(f'Result for {video_path}', result_frame)
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break
    
    cap.release()
    out.release()
    cv2.destroyAllWindows()

def process_folder(predictor, folder_path, ramp_index):
    for file_name in os.listdir(folder_path):
        file_path = os.path.join(folder_path, file_name)
        if os.path.isfile(file_path):
            if file_path.lower().endswith(('.png', '.jpg', '.jpeg', '.bmp', '.tif', '.tiff')):
                process_image(predictor, file_path, ramp_index)
            elif file_path.lower().endswith(('.mp4', '.avi', '.mov', '.mkv')):
                output_path = os.path.join(folder_path, f"processed_{file_name}")
                process_video(predictor, file_path, ramp_index, output_path)

def main():
    config_file = "C:/Users/Daanish Mittal/OneDrive/Desktop/Tank_align/tank_alignment/army_ramp/loader.ramp/mask_rcnn_R_101_FPN_3x/2024-07-09-01-55-42/config.yaml"
    weights_file = "C:/Users/Daanish Mittal/OneDrive/Desktop/Tank_align/tank_alignment/army_ramp/loader.ramp/mask_rcnn_R_101_FPN_3x/2024-07-09-01-55-42/model_final.pth"
    input_folder = "C:/Users/Daanish Mittal/OneDrive/Desktop/Tank_align/tank_alignment/test_army"

    predictor = load_model(config_file, weights_file)
    
    # Set ramp_index directly
    ramp_index = 1
    
    process_folder(predictor, input_folder, ramp_index)

if __name__ == "__main__":
    main()

In [13]:
import os
import cv2
import numpy as np
from detectron2.config import get_cfg
from detectron2.engine import DefaultPredictor

def load_model(config_file, weights_file):
    cfg = get_cfg()
    cfg.merge_from_file(config_file)
    cfg.MODEL.WEIGHTS = weights_file
    cfg.MODEL.ROI_HEADS.SCORE_THRESH_TEST = 0.99
    return DefaultPredictor(cfg)

def mask_sides(image, mask_percentage=0.2):
    h, w = image.shape[:2]
    mask_width = int(w * mask_percentage)
    
    mask = np.ones(image.shape[:2], dtype=np.uint8)
    mask[:, :mask_width] = 0
    mask[:, -mask_width:] = 0
    
    masked_image = cv2.bitwise_and(image, image, mask=mask)
    
    return masked_image, mask

def tint_screen(image, color):
    overlay = np.zeros_like(image)
    output = image.copy()
    alpha = 0.3  # Transparency factor.
    if color == 'green':
        overlay[:] = (0, 255, 0)
    elif color == 'orange':
        overlay[:] = (0, 165, 255)
    else:
        overlay[:] = (0, 0, 255)
    cv2.addWeighted(overlay, alpha, output, 1 - alpha, 0, output)
    return output

def calculate_angle(vector1, vector2):
    # Ensure vectors point upwards
    if vector1[1] > 0:
        vector1 = -vector1
    if vector2[1] > 0:
        vector2 = -vector2
    
    # Calculate unit vectors
    unit_vector1 = vector1 / np.linalg.norm(vector1)
    unit_vector2 = vector2 / np.linalg.norm(vector2)
    
    # Calculate dot product and angle
    dot_product = np.dot(unit_vector1, unit_vector2)
    angle = np.arccos(np.clip(dot_product, -1.0, 1.0))
    
    return np.degrees(angle)

def calculate_distance(point, line_x):
    return abs(point[0] - line_x)

def process_frame(predictor, frame, ramp_index):
    masked_frame, frame_mask = mask_sides(frame, 0.2)
    
    outputs = predictor(masked_frame)
    instances = outputs["instances"].to("cpu")
    ramp_instances = instances[instances.pred_classes == ramp_index]
    
    draw_frame = masked_frame.copy()
    
    for i in range(len(ramp_instances)):
        if ramp_instances.has("pred_masks"):
            mask = ramp_instances.pred_masks[i].numpy()
            mask = cv2.bitwise_and(mask.astype(np.uint8), frame_mask)
            
            contours, _ = cv2.findContours(mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
            if contours:
                largest_contour = max(contours, key=cv2.contourArea)
                
                epsilon = 0.05 * cv2.arcLength(largest_contour, True)
                approx_contour = cv2.approxPolyDP(largest_contour, epsilon, True)
                
                if len(approx_contour) > 4:
                    approx_contour = cv2.convexHull(approx_contour)
                    approx_contour = cv2.approxPolyDP(approx_contour, epsilon, True)
                
                if len(approx_contour) > 4:
                    approx_contour = cv2.approxPolyDP(approx_contour, epsilon * 2, True)
                
                if len(approx_contour) > 4:
                    approx_contour = cv2.convexHull(approx_contour)
                    approx_contour = cv2.approxPolyDP(approx_contour, epsilon * 3, True)
                
                approx_contour = approx_contour[:4]

                # Find the rightmost edge of the polygon
                rightmost_points = sorted(approx_contour, key=lambda x: x[0][0])[-2:]
                right_edge_start = tuple(rightmost_points[0][0])
                right_edge_end = tuple(rightmost_points[1][0])

                # Ensure the edge is drawn from top to bottom
                if right_edge_start[1] > right_edge_end[1]:
                    right_edge_start, right_edge_end = right_edge_end, right_edge_start

                cv2.line(draw_frame, right_edge_start, right_edge_end, (128, 0, 128), 2)
                
                # Draw the vertical green line in the middle of the frame
                mid_x = draw_frame.shape[1] // 2
                cv2.line(draw_frame, (mid_x, 0), (mid_x, draw_frame.shape[0]), (0, 255, 0), 2)

                # Calculate the vector of the right edge
                vector_right_edge = np.array([right_edge_end[0] - right_edge_start[0], right_edge_end[1] - right_edge_start[1]])
                
                # Calculate the vector of the green line (always vertical)
                vector_green_line = np.array([0, -1])  # Pointing upwards

                # Calculate the angle between the right edge and the green line
                angle_degrees = calculate_angle(vector_right_edge, vector_green_line)

                # Calculate the distance between the green line and the closest point on the right edge
                distances = [calculate_distance(point[0], mid_x) for point in rightmost_points]
                min_distance = min(distances)

                # Apply tint based on angle and distance
                if angle_degrees < 5:
                    if min_distance < 5:
                        draw_frame = tint_screen(draw_frame, 'green')
                    elif min_distance < 15:
                        draw_frame = tint_screen(draw_frame, 'orange')
                    else:
                        draw_frame = tint_screen(draw_frame, 'red')
                elif angle_degrees < 15:
                    if min_distance < 15:
                        draw_frame = tint_screen(draw_frame, 'orange')
                    else:
                        draw_frame = tint_screen(draw_frame, 'red')
                else:
                    draw_frame = tint_screen(draw_frame, 'red')

                # Draw angle information
                x, y, w, h = cv2.boundingRect(largest_contour)
                cv2.putText(draw_frame, f"Angle: {angle_degrees:.2f}", (x, y - 10), 
                            cv2.FONT_HERSHEY_SIMPLEX, 0.9, (0, 255, 255), 2)
    
    return draw_frame

def process_video(predictor, video_path, output_path, ramp_index):
    cap = cv2.VideoCapture(video_path)
    fourcc = cv2.VideoWriter_fourcc(*'mp4v')
    frame_width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
    frame_height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
    fps = int(cap.get(cv2.CAP_PROP_FPS))
    out = cv2.VideoWriter(output_path, fourcc, fps, (frame_width, frame_height))

    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break
        
        result_frame = process_frame(predictor, frame, ramp_index)
        out.write(result_frame)
        
        cv2.imshow('Processed Video', result_frame)
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break
    
    cap.release()
    out.release()
    cv2.destroyAllWindows()

def process_folder(predictor, input_folder, ramp_index):
    for filename in os.listdir(input_folder):
        if filename.endswith(('.mp4', '.avi', '.mov')):
            input_path = os.path.join(input_folder, filename)
            output_path = os.path.join(input_folder, f'processed_{filename}')
            process_video(predictor, input_path, output_path, ramp_index)

def main():
    config_file = "C:/Users/Daanish Mittal/OneDrive/Desktop/Tank_align/tank_alignment/army_ramp/loader.ramp/mask_rcnn_R_101_FPN_3x/2024-07-09-01-55-42/config.yaml"
    weights_file = "C:/Users/Daanish Mittal/OneDrive/Desktop/Tank_align/tank_alignment/army_ramp/loader.ramp/mask_rcnn_R_101_FPN_3x/2024-07-09-01-55-42/model_final.pth"
    input_folder = "C:/Users/Daanish Mittal/OneDrive/Desktop/Tank_align/tank_alignment/test_army"

    predictor = load_model(config_file, weights_file)
    
    # Set ramp_index directly
    ramp_index = 1
    
    process_folder(predictor, input_folder, ramp_index)

if __name__ == "__main__":
    main()

INFO:detectron2.checkpoint.detection_checkpoint:[DetectionCheckpointer] Loading from C:/Users/Daanish Mittal/OneDrive/Desktop/Tank_align/tank_alignment/army_ramp/loader.ramp/mask_rcnn_R_101_FPN_3x/2024-07-09-01-55-42/model_final.pth ...
INFO:fvcore.common.checkpoint:[Checkpointer] Loading from c:/Users/Daanish Mittal/OneDrive/Desktop/Tank_align/tank_alignment/army_ramp/loader.ramp/mask_rcnn_R_101_FPN_3x/2024-07-09-01-55-42/model_final.pth ...


***Ramp with Webcam***  

In [4]:
import os
import cv2
import torch
from collections import deque
from detectron2.config import get_cfg
from detectron2.engine import DefaultPredictor
from detectron2.utils.visualizer import Visualizer
from detectron2.data import MetadataCatalog
import numpy as np

# Constants
MASK_PERCENTAGE = 0.2
TRANSPARENCY_ALPHA = 0.3
GREEN_COLOR = (0, 255, 0)
ORANGE_COLOR = (0, 165, 255)
RED_COLOR = (0, 0, 255)
EDGE_COLOR = (128, 0, 128)
LINE_THICKNESS = 2
SMOOTHING_EPSILON_FACTOR = 0.05
FRAME_HISTORY = 5

# Deque to store edge positions
edge_history = deque(maxlen=FRAME_HISTORY)

def load_model(config_file, weights_file):
    print("Loading model...")
    cfg = get_cfg()
    cfg.merge_from_file(config_file)
    cfg.MODEL.WEIGHTS = weights_file
    cfg.MODEL.ROI_HEADS.SCORE_THRESH_TEST = 0.97  # Set a custom testing threshold
    print("Model loaded successfully.")
    return DefaultPredictor(cfg)

def mask_sides(image, mask_percentage):
    h, w = image.shape[:2]
    mask_width = int(w * mask_percentage)
    mask = np.ones(image.shape[:2], dtype=np.uint8)
    mask[:, :mask_width] = 0
    mask[:, -mask_width:] = 0
    masked_image = cv2.bitwise_and(image, image, mask=mask)
    return masked_image, mask

def tint_screen(image, color, alpha):
    overlay = np.zeros_like(image)
    if color == 'green':
        overlay[:] = GREEN_COLOR
    elif color == 'orange':
        overlay[:] = ORANGE_COLOR
    else:
        overlay[:] = RED_COLOR
    cv2.addWeighted(overlay, alpha, image, 1 - alpha, 0, image)
    return image

def calculate_angle(vector1, vector2):
    if vector1[1] > 0:
        vector1 = -vector1
    if vector2[1] > 0:
        vector2 = -vector2
    unit_vector1 = vector1 / np.linalg.norm(vector1)
    unit_vector2 = vector2 / np.linalg.norm(vector2)
    dot_product = np.dot(unit_vector1, unit_vector2)
    angle = np.arccos(np.clip(dot_product, -1.0, 1.0))
    return np.degrees(angle)

def calculate_distance(point, line_x):
    return abs(point[0] - line_x)

def find_right_edge(contour):
    epsilon = SMOOTHING_EPSILON_FACTOR * cv2.arcLength(contour, True)
    approx_contour = cv2.approxPolyDP(contour, epsilon, True)
    if len(approx_contour) > 4:
        approx_contour = cv2.convexHull(approx_contour)
        approx_contour = cv2.approxPolyDP(approx_contour, epsilon, True)
    approx_contour = approx_contour[:4]
    rightmost_points = sorted(approx_contour, key=lambda x: x[0][0])[-2:]
    right_edge_start, right_edge_end = sorted(rightmost_points, key=lambda x: x[0][1])
    return right_edge_start[0], right_edge_end[0]

def average_edge_position(edge_history):
    avg_start = np.mean([edge[0] for edge in edge_history], axis=0).astype(int)
    avg_end = np.mean([edge[1] for edge in edge_history], axis=0).astype(int)
    return tuple(avg_start), tuple(avg_end)

def process_frame(frame, predictor, mask_percentage):
    masked_frame, frame_mask = mask_sides(frame, mask_percentage)
    outputs = predictor(masked_frame)
    instances = outputs["instances"].to("cpu")
    ramp_instances = instances[instances.pred_classes == 1]
    draw_frame = masked_frame.copy()
    for i in range(len(ramp_instances)):
        if ramp_instances.has("pred_masks"):
            mask = ramp_instances.pred_masks[i].numpy()
            mask = cv2.bitwise_and(mask.astype(np.uint8), frame_mask)
            contours, _ = cv2.findContours(mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
            if contours:
                largest_contour = max(contours, key=cv2.contourArea)
                right_edge_start, right_edge_end = find_right_edge(largest_contour)
                edge_history.append((right_edge_start, right_edge_end))
                if len(edge_history) == FRAME_HISTORY:
                    right_edge_start, right_edge_end = average_edge_position(edge_history)
                cv2.line(draw_frame, right_edge_start, right_edge_end, EDGE_COLOR, LINE_THICKNESS)
                
                mid_x = draw_frame.shape[1] // 2
                frame_height = draw_frame.shape[0]
                green_line_start = int(frame_height * 0.28)  # Starting point of the green line (28% from the top)
                green_line_end = frame_height  # Ending point of the green line (bottom of the frame)
                cv2.line(draw_frame, (mid_x, green_line_start), (mid_x, green_line_end), GREEN_COLOR, LINE_THICKNESS)
                
                vector_right_edge = np.array([right_edge_end[0] - right_edge_start[0], right_edge_end[1] - right_edge_start[1]])
                vector_green_line = np.array([0, -1])
                angle_degrees = calculate_angle(vector_right_edge, vector_green_line)
                distances = [calculate_distance(point, mid_x) for point in [right_edge_start, right_edge_end]]
                min_distance = min(distances)
                if angle_degrees < 5:
                    if min_distance < 5:
                        draw_frame = tint_screen(draw_frame, 'green', TRANSPARENCY_ALPHA)
                    elif min_distance < 15:
                        draw_frame = tint_screen(draw_frame, 'orange', TRANSPARENCY_ALPHA)
                    else:
                        draw_frame = tint_screen(draw_frame, 'red', TRANSPARENCY_ALPHA)
                elif angle_degrees < 15:
                    if min_distance < 15:
                        draw_frame = tint_screen(draw_frame, 'orange', TRANSPARENCY_ALPHA)
                    else:
                        draw_frame = tint_screen(draw_frame, 'red', TRANSPARENCY_ALPHA)
                else:
                    draw_frame = tint_screen(draw_frame, 'red', TRANSPARENCY_ALPHA)
    return draw_frame

def process_video(video_path, predictor):
    cap = cv2.VideoCapture(video_path)
    width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
    height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
    fps = int(cap.get(cv2.CAP_PROP_FPS))
    fourcc = cv2.VideoWriter_fourcc(*'mp4v')
    out = cv2.VideoWriter('thisisoutput005.mp4', fourcc, fps, (width, height))
    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break
        processed_frame = process_frame(frame, predictor, MASK_PERCENTAGE)
        out.write(processed_frame)
    cap.release()
    out.release()

# Example usage
config_file = "C:/Users/Daanish Mittal/OneDrive/Desktop/Tank_align/tank_alignment/army_ramp/loader.ramp/mask_rcnn_R_101_FPN_3x/2024-07-09-01-55-42/config.yaml"
weights_file = "C:/Users/Daanish Mittal/OneDrive/Desktop/Tank_align/tank_alignment/army_ramp/loader.ramp/mask_rcnn_R_101_FPN_3x/2024-07-09-01-55-42/model_final.pth"
video_path = "C:/Users/Daanish Mittal/OneDrive/Desktop/Tank_align/tank_alignment/test_army/WhatsApp Video 2024-07-18 at 15.22.47_a6e64e0a.mp4"

predictor = load_model(config_file, weights_file)
process_video(video_path, predictor)


Loading model...
Model loaded successfully.


KeyboardInterrupt: 

In [ ]:
import torch
torch.cuda.set_device(0)  # Use the first GPU

In [ ]:
import os
import cv2
import torch
from collections import deque
from detectron2.config import get_cfg
from detectron2.engine import DefaultPredictor
from detectron2.utils.visualizer import Visualizer
from detectron2.data import MetadataCatalog
import numpy as np

# Constants
MASK_PERCENTAGE = 0.2
TRANSPARENCY_ALPHA = 0.3
GREEN_COLOR = (0, 255, 0)
ORANGE_COLOR = (0, 165, 255)
RED_COLOR = (0, 0, 255)
YELLOW_COLOR = (0, 255, 255)  # Changed from violet to yellow
LINE_THICKNESS = 2
SMOOTHING_EPSILON_FACTOR = 0.05
FRAME_HISTORY = 5
DESIRED_FPS = 5

# Deque to store edge positions
edge_history = deque(maxlen=FRAME_HISTORY)

def load_model(config_file, weights_file):
    print("Loading model...")
    cfg = get_cfg()
    cfg.merge_from_file(config_file)
    cfg.MODEL.WEIGHTS = weights_file
    cfg.MODEL.ROI_HEADS.SCORE_THRESH_TEST = 0.97  # Set a custom testing threshold
    print("Model loaded successfully.")
    return DefaultPredictor(cfg)

def mask_sides(image, mask_percentage):
    h, w = image.shape[:2]
    mask_width = int(w * mask_percentage)
    mask = np.ones(image.shape[:2], dtype=np.uint8)
    mask[:, :mask_width] = 0
    mask[:, -mask_width:] = 0
    masked_image = cv2.bitwise_and(image, image, mask=mask)
    return masked_image, mask

def tint_screen(image, color, alpha):
    overlay = np.zeros_like(image)
    if color == 'green':
        overlay[:] = GREEN_COLOR
    elif color == 'orange':
        overlay[:] = ORANGE_COLOR
    else:
        overlay[:] = RED_COLOR
    cv2.addWeighted(overlay, alpha, image, 1 - alpha, 0, image)
    return image

def calculate_angle(vector1, vector2):
    if vector1[1] > 0:
        vector1 = -vector1
    if vector2[1] > 0:
        vector2 = -vector2
    unit_vector1 = vector1 / np.linalg.norm(vector1)
    unit_vector2 = vector2 / np.linalg.norm(vector2)
    dot_product = np.dot(unit_vector1, unit_vector2)
    angle = np.arccos(np.clip(dot_product, -1.0, 1.0))
    return np.degrees(angle)

def calculate_distance(point, line_x):
    return abs(point[0] - line_x)

def find_right_edge(contour):
    epsilon = SMOOTHING_EPSILON_FACTOR * cv2.arcLength(contour, True)
    approx_contour = cv2.approxPolyDP(contour, epsilon, True)
    if len(approx_contour) > 4:
        approx_contour = cv2.convexHull(approx_contour)
        approx_contour = cv2.approxPolyDP(approx_contour, epsilon, True)
    approx_contour = approx_contour[:4]
    rightmost_points = sorted(approx_contour, key=lambda x: x[0][0])[-2:]
    right_edge_start, right_edge_end = sorted(rightmost_points, key=lambda x: x[0][1])
    return right_edge_start[0], right_edge_end[0]

def average_edge_position(edge_history):
    avg_start = np.mean([edge[0] for edge in edge_history], axis=0).astype(int)
    avg_end = np.mean([edge[1] for edge in edge_history], axis=0).astype(int)
    return tuple(avg_start), tuple(avg_end)

def process_frame(frame, predictor, mask_percentage):
    masked_frame, frame_mask = mask_sides(frame, mask_percentage)
    outputs = predictor(masked_frame)
    instances = outputs["instances"].to("cpu")
    ramp_instances = instances[instances.pred_classes == 1]
    draw_frame = masked_frame.copy()
    for i in range(len(ramp_instances)):
        if ramp_instances.has("pred_masks"):
            mask = ramp_instances.pred_masks[i].numpy()
            mask = cv2.bitwise_and(mask.astype(np.uint8), frame_mask)
            contours, _ = cv2.findContours(mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
            if contours:
                largest_contour = max(contours, key=cv2.contourArea)
                
                # Highlight the detected object
                cv2.drawContours(draw_frame, [largest_contour], 0, YELLOW_COLOR, 2)
                
                right_edge_start, right_edge_end = find_right_edge(largest_contour)
                edge_history.append((right_edge_start, right_edge_end))
                if len(edge_history) == FRAME_HISTORY:
                    right_edge_start, right_edge_end = average_edge_position(edge_history)
                cv2.line(draw_frame, right_edge_start, right_edge_end, YELLOW_COLOR, LINE_THICKNESS)
                
                mid_x = draw_frame.shape[1] // 2
                frame_height = draw_frame.shape[0]
                green_line_start = int(frame_height * 0.28)  # Starting point of the green line (28% from the top)
                green_line_end = frame_height  # Ending point of the green line (bottom of the frame)
                cv2.line(draw_frame, (mid_x, green_line_start), (mid_x, green_line_end), GREEN_COLOR, LINE_THICKNESS)
                
                vector_right_edge = np.array([right_edge_end[0] - right_edge_start[0], right_edge_end[1] - right_edge_start[1]])
                vector_green_line = np.array([0, -1])
                angle_degrees = calculate_angle(vector_right_edge, vector_green_line)
                distances = [calculate_distance(point, mid_x) for point in [right_edge_start, right_edge_end]]
                min_distance = min(distances)
                if angle_degrees < 5:
                    if min_distance < 5:
                        draw_frame = tint_screen(draw_frame, 'green', TRANSPARENCY_ALPHA)
                    elif min_distance < 15:
                        draw_frame = tint_screen(draw_frame, 'orange', TRANSPARENCY_ALPHA)
                    else:
                        draw_frame = tint_screen(draw_frame, 'red', TRANSPARENCY_ALPHA)
                elif angle_degrees < 15:
                    if min_distance < 15:
                        draw_frame = tint_screen(draw_frame, 'orange', TRANSPARENCY_ALPHA)
                    else:
                        draw_frame = tint_screen(draw_frame, 'red', TRANSPARENCY_ALPHA)
                else:
                    draw_frame = tint_screen(draw_frame, 'red', TRANSPARENCY_ALPHA)
    return draw_frame


def process_video(video_path, predictor):
    cap = cv2.VideoCapture(video_path)
    width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
    height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
    original_fps = int(cap.get(cv2.CAP_PROP_FPS))
    frame_interval = max(1, original_fps // DESIRED_FPS)
    frame_count = 0

    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break
        
        # Skip frames to achieve the desired fps
        if frame_count % frame_interval == 0:
            processed_frame = process_frame(frame, predictor, MASK_PERCENTAGE)
            
            # Resize frame to 70% of original size
            resized_frame = cv2.resize(processed_frame, (int(width * 0.7), int(height * 0.7)))
            
            # Display the resulting frame
            cv2.imshow('Processed Video', resized_frame)
            
            # Press 'q' to quit
            if cv2.waitKey(1) & 0xFF == ord('q'):
                break
        
        frame_count += 1

    cap.release()
    cv2.destroyAllWindows()

# Example usage
config_file = "C:/Users/Daanish Mittal/OneDrive/Desktop/Tank_align/tank_alignment/army_ramp/loader.ramp/mask_rcnn_R_101_FPN_3x/2024-07-09-01-55-42/config.yaml"
weights_file = "C:/Users/Daanish Mittal/OneDrive/Desktop/Tank_align/tank_alignment/army_ramp/loader.ramp/mask_rcnn_R_101_FPN_3x/2024-07-09-01-55-42/model_final.pth"
video_path = "C:/Users/Daanish Mittal/OneDrive/Desktop/Tank_align/tank_alignment/test_army/WhatsApp Video 2024-07-18 at 15.22.47_a6e64e0a.mp4"

predictor = load_model(config_file, weights_file)
process_video(video_path, predictor)


Loading model...
Model loaded successfully.


****ramp with ipcam****